## 1️⃣ Reading a .pose File in Python (Basic Method)

In [1]:
from pose_format import Pose

In [2]:
path_pose = '../data/pose_files/example.pose'
save_path = "../output/read_pose_files/"

In [3]:
def read_pose_basic(pose_path: str) -> Pose:
    """
    Basic method to read a .pose file
    Uses NumPy backend by default
    """
    with open(pose_path, 'rb') as f:
        buffer = f.read()
    
    pose = Pose.read(buffer)
    return pose


In [4]:
# Example usage
pose = read_pose_basic(path_pose)
print(f"Number of frames: {pose.body.data.shape[0]}")
print(f"Number of points: {pose.body.data.shape[2]}")

print(f"✅ Loaded as NumPy array")
print(f"   Data type: {type(pose.body.data)}")
print(f"   Shape: {pose.body.data.shape}")


Number of frames: 133
Number of points: 203
✅ Loaded as NumPy array
   Data type: <class 'numpy.ma.core.MaskedArray'>
   Shape: (133, 1, 203, 3)


In [5]:
from io import BytesIO

def read_pose_from_bytes(data: bytes) -> Pose:
    """
    Read directly from bytes (useful when downloading from the internet)
    """
    return Pose.read(data)

def read_pose_from_bytesio(buffer: BytesIO) -> Pose:
    """
    Read from BytesIO (useful for streaming)
    """
    return Pose.read(buffer)


In [6]:
# 🧪 Test reading from bytes
with open(path_pose, 'rb') as f:
    file_bytes = f.read()

# Direct read from bytes
pose_from_bytes = read_pose_from_bytes(file_bytes)
print(f"✅ Read from bytes")
print(f"   Shape: {pose_from_bytes.body.data.shape}")

# Read from BytesIO
from io import BytesIO
buffer = BytesIO(file_bytes)
pose_from_buffer = read_pose_from_bytesio(buffer)
print(f"✅ Read from BytesIO")
print(f"   Same result? {pose_from_buffer.body.data.shape == pose_from_bytes.body.data.shape}")

✅ Read from bytes
   Shape: (133, 1, 203, 3)
✅ Read from BytesIO
   Same result? True


## 2️⃣ Reading a specific part of the file (for large files)

In [7]:
from io import BytesIO

def read_pose_partial(pose_path: str, 
                    start_frame: int = None, 
                    end_frame: int = None) -> Pose:
    """
    Read a partial segment from a .pose file.
    Very useful for large files!
    
    Parameters:
    -----------
    pose_path : str
        Path to the file
    start_frame : int, optional
        Index of the first frame
    end_frame : int, optional
        Index of the last frame
    """
    with open(pose_path, 'rb') as f:
        buffer = BytesIO(f.read())
    
    pose = Pose.read(
        buffer,
        start_frame=start_frame,
        end_frame=end_frame
    )
    
    return pose


In [8]:
def save_as_pose(pose: Pose, output_path: str):
    """
    Saves a Pose object to a .pose file.
    """
    with open(output_path, 'wb') as f:
        pose.write(f)
    print(f"✅ Saved to: {output_path}")


In [9]:
# Examples:
# First 100 frames only
pose = read_pose_partial(path_pose, start_frame=0, end_frame=100)
save_as_pose(pose, f"{save_path}/output_0_100.pose")

✅ Saved to: ../output/read_pose_files//output_0_100.pose


In [10]:
# First 50 frames only
pose = read_pose_partial(path_pose, start_frame=0, end_frame=50)
save_as_pose(pose, f"{save_path}/output_0_50.pose")

✅ Saved to: ../output/read_pose_files//output_0_50.pose


In [11]:
# First 50 frames only
pose = read_pose_partial(path_pose, start_frame=0, end_frame=200)
save_as_pose(pose, f"{save_path}/output_0_200.pose")

✅ Saved to: ../output/read_pose_files//output_0_200.pose


## 3️⃣ Read Header Only (without data)

In [12]:
from pose_format.pose_body import EmptyPoseBody
from io import BytesIO

def read_pose_header_only(pose_path: str):
    """
    Read header only without loading the data
    Useful for getting file information quickly
    """
    with open(pose_path, 'rb') as f:
        buffer = BytesIO(f.read())
    
    pose = Pose.read(buffer, pose_body=EmptyPoseBody)
    
    print(f"File information:")
    print(f"  Version: {pose.header.version}")
    print(f"  Dimensions: {pose.header.dimensions.width}x{pose.header.dimensions.height}")
    print(f"  Components count: {len(pose.header.components)}")
    print(f"  Total points: {pose.header.total_points()}")
    
    for comp in pose.header.components:
        print(f"    - {comp.name}: {len(comp.points)} points")
    
    return pose.header



In [13]:
header = read_pose_header_only(path_pose)

File information:
  Version: 0.20000000298023224
  Dimensions: 640x480
  Components count: 4
  Total points: 203
    - POSE_LANDMARKS: 33 points
    - FACE_LANDMARKS: 128 points
    - LEFT_HAND_LANDMARKS: 21 points
    - RIGHT_HAND_LANDMARKS: 21 points


## 4️⃣ Using PyTorch Backend

In [ ]:
# Install PyTorch if not already installed
# !pip install torch

In [14]:
def read_pose_pytorch(pose_path: str):
    """
    Read .pose file with PyTorch backend
    Useful for training and inference with PyTorch models
    """
    try:
        from pose_format.torch.pose_body import TorchPoseBody
    except ImportError:
        print("❌ PyTorch not installed. Install with: pip install torch")
        return None
    
    with open(pose_path, 'rb') as f:
        buffer = f.read()
    
    # Read directly as TorchPoseBody
    pose = Pose.read(buffer, pose_body=TorchPoseBody)
    
    print(f"✅ Loaded as PyTorch tensors")
    print(f"   Data type: {type(pose.body.data)}")
    print(f"   Shape: {pose.body.data.shape}")
    print(f"   Device: {pose.body.data.device}")
    
    return pose



In [15]:
pose_torch = read_pose_pytorch(path_pose)

✅ Loaded as PyTorch tensors
   Data type: <class 'pose_format.torch.masked.tensor.MaskedTensor'>
   Shape: torch.Size([133, 1, 203, 3])
   Device: cpu


In [ ]:
# Move to GPU if available
import torch
if pose_torch is not None and torch.cuda.is_available():
    pose_torch.body.cuda()

In [ ]:
print(pose_torch.body.data.device)

In [16]:
print(type(pose_torch.body.data))            # Should be torch.Tensor
print(pose_torch.body.data.device)           # "cuda:0"
print(pose_torch.body.data.is_cuda)          # True

<class 'pose_format.torch.masked.tensor.MaskedTensor'>
cpu
False


## 5️⃣ Using TensorFlow Backend

In [ ]:
# Install TensorFlow if not already installed
# !pip install tensorflow

In [17]:
def read_pose_tensorflow(pose_path: str):
    """
    Read .pose file with TensorFlow backend
    """
    try:
        from pose_format.tensorflow.pose_body import TensorflowPoseBody
    except ImportError:
        print("❌ TensorFlow not installed. Install with: pip install tensorflow")
        return None
    
    with open(pose_path, 'rb') as f:
        buffer = f.read()
    
    pose = Pose.read(buffer, pose_body=TensorflowPoseBody)
    
    print(f"✅ Loaded as TensorFlow tensors")
    print(f"   Data type: {type(pose.body.data)}")
    print(f"   Shape: {pose.body.data.shape}")
    
    return pose



In [18]:
pose_tf = read_pose_tensorflow(path_pose)

❌ TensorFlow not installed. Install with: pip install tensorflow


In [ ]:
def numpy_to_tensorflow(pose: Pose):
    """
    Convert from NumPy to TensorFlow
    """
    return pose.tensorflow()

pose_numpy = read_pose_basic(path_pose)
pose_tf = numpy_to_tensorflow(pose_numpy)

## 6️⃣ Reading from URL in Python

In [19]:
import requests

def read_pose_from_url(url: str) -> Pose:
    """
    Download .pose file from URL
    """
    response = requests.get(url)
    response.raise_for_status()
    
    pose = Pose.read(response.content)
    
    print(f"✅ Downloaded from: {url}")
    print(f"   Frames: {pose.body.data.shape[0]}")
    print(f"   Points: {pose.body.data.shape[2]}")
    
    return pose



In [20]:
file_url = "https://github.com/24-mohamedyehia/pose-format-tutorials/raw/refs/heads/main/data/pose_files/example.pose"
pose = read_pose_from_url(file_url)

✅ Downloaded from: https://github.com/24-mohamedyehia/pose-format-tutorials/raw/refs/heads/main/data/pose_files/example.pose
   Frames: 133
   Points: 203


## 📚 Summary

### Python
| Method | Usage |
|---------|----------|
| `Pose.read(buffer)` | Basic method (NumPy) |
| `Pose.read(buffer, TorchPoseBody)` | For training with PyTorch |
| `Pose.read(buffer, TensorflowPoseBody)` | For training with TensorFlow |
| `pose.torch()` | Convert to PyTorch |
| `pose.tensorflow()` | Convert to TensorFlow |

### 🎯 Next Step
Move to the next Notebook to learn how to **visualize and display poses**!